In [15]:
# Dependencies
import tweepy
import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Twitter API Keys
consumer_key = "sz5wb5oRXgH9bnTIZFSgvhv3b"
consumer_secret = "OrT3JaaW3JtzMeno4Q6X6bDYC5xyrAp2MERVHR6oxubdnpkVHv"
access_token = "979169790735470592-YZh7iRKW87dk7Ce7IAYlzGITHNC66hI"
access_token_secret = "jW0whjf3UGwf6BuOnOdmwLFB7BCrOJjI2WdI0srIoeRKg"

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [16]:
target1 = "@cnn"
tweet_test = api.search(target1,count=3,result_type="recent")
print(json.dumps(tweet,sort_keys=True,indent=4))

{
    "contributors": null,
    "coordinates": null,
    "created_at": "Sun Apr 08 05:29:07 +0000 2018",
    "entities": {
        "hashtags": [],
        "symbols": [],
        "urls": [
            {
                "display_url": "bbc.in/2qfDIh0",
                "expanded_url": "https://bbc.in/2qfDIh0",
                "indices": [
                    82,
                    105
                ],
                "url": "https://t.co/oSdO2X8IAX"
            }
        ],
        "user_mentions": [
            {
                "id": 742143,
                "id_str": "742143",
                "indices": [
                    3,
                    12
                ],
                "name": "BBC News (World)",
                "screen_name": "BBCWorld"
            }
        ]
    },
    "favorite_count": 0,
    "favorited": false,
    "geo": null,
    "id": 982852819760963584,
    "id_str": "982852819760963584",
    "in_reply_to_screen_name": null,
    "in_reply_to_status_id": null,

In [17]:
# create list of twitter handles for news sources to target
target_handles = ("@BBCWorld", "@CBS", "@CNN",
                "@FoxNews", "@nytimes")

#set filters for "real tweets"
min_tweets = 5
max_tweets = 10000
max_followers = 2500
max_following = 2500
lang = "en"


#loop through all target users
for target in target_handles:
    
    #variable for oldest tweet
    oldest_tweet = None
    
    #variables for sentiments/user
    #target = []
    compound_list=[]
    positive_list = []
    negative_list = []
    neutral_list = []
    target_list = []
    tweet_count = []
    counter=1
    # loop through 5x (20 tweets per page)
    for x in range(5):
        
        #run search through each tweet
        public_tweets = api.search(target,count=100,result_type="recent",max_id = oldest_tweet)
        
        #loop through all tweets
        for tweet in public_tweets["statuses"]:

         # Use filters to check if user meets conditions
            if (tweet["user"]["followers_count"] < max_followers
                and tweet["user"]["statuses_count"] > min_tweets
                and tweet["user"]["statuses_count"] < max_tweets
                and tweet["user"]["friends_count"] < max_following
                and tweet["user"]["lang"] == lang):
                
            # Run Vader Analysis on each tweet
                results = analyzer.polarity_scores(tweet["text"])
                compound = results["compound"]
                pos = results["pos"]
                neu = results["neu"]
                neg = results["neg"]
             
            #create variables for target and tweet count
                target= tweet["user"]
                tweets_ago = counter
                
                # Add each value to the appropriate list
                compound_list.append(compound)
                positive_list.append(pos)
                negative_list.append(neg)
                neutral_list.append(neu)
                target_list.append(target)
                tweet_count.append(tweets_ago)
            
            # Set the new oldest_tweet value and counter of tweet
            oldest_tweet = tweet["id"] - 1
            counter = counter+1
        
       

TweepError: [{'code': 195, 'message': 'Missing or invalid url parameter.'}]

In [ ]:
#add looped data into dataframe

results_df = pd.DataFrame({"User":target_list,"Compound":compound_list,"Neutral":neutral_list,"Positive":positive_list,"Negative":negative_list,"Tweets Ago":tweets_ago})
results_df


In [ ]:
#Sentiment Over Last 100 Tweets
#create seperate data frames for each handle then plot data frame in scatter plot
compound_df = results_df[["User","Compound"]]
tweetsago_df = results_df[["User","Tweets Ago"]]

BBC_compound = compound_df[compound_df["User"]=="BBCWorld"]
BBC_tweets = tweetsago_df[tweetsago_df["User"]=="BBCWorld"]
plt.scatter(BBC_tweets, BBC_compound, c = "red", edgecolor="black", linewidths=1, marker = "o", alpha=0.8, label="BBC")

CBS_compound = compound_df[compound_df["User"]=="CBS"]
CBS_tweets = tweetsago_df[tweetsago_df["User"]=="CBS"]
plt.scatter(CBS_tweets, CBS_compound, c = "orange", edgecolor="black", linewidths=1, marker = "o", alpha=0.8, label="CBS")

CNN_compound = compound_df[compound_df["User"]=="CNN"]
CNN_tweets = tweetsago_df[tweetsago_df["User"]=="CNN"]
plt.scatter(CNN_tweets, CNN_compound, c = "yellow", edgecolor="black", linewidths=1, marker = "o", alpha=0.8, label="CNN")

Fox_compound = compound_df[compound_df["User"]=="FoxNews"]
Fox_tweets = tweetsago_df[tweetsago_df["User"]=="FoxNews"]
plt.scatter(Fox_tweets, Fox_compound, c = "green", edgecolor="black", linewidths=1, marker = "o", alpha=0.8, label="Fox")

nytimes_compound = compound_df[compound_df["User"]=="nytimes"]
nytimes_tweets = tweetsago_df[tweetsago_df["User"]=="nytimes"]
plt.scatter(nytimes_tweets, nytimes_compound, c = "blue", edgecolor="black", linewidths=1, marker = "o", alpha=0.8, label="NY Times")

#add graph elements
plt.title("Sentiment Analysis of Media Tweets")
ply.ylabel("Tweet Polarity")
plt.xlabel("Number of Tweets Ago")
plt.xlim(0,100)
plt.ylim(-1,1)
plt.grid(True)

#create a legend
legend = plt.legend(fontsize="small",mode = "Expanded",numpoints=1,scatterpoints=1,loc="best",title="Media Sources",labelspacing=0.5)
plt.savefig("sentiment.png")

#show plot
plt.show()

In [ ]:
#Overall Sentiment of media sources
bar_df = results_df[["User","Compound"]]
bar_df= bar_df.groupby(["User"]).mean()["Compound"]
users=["@BBCWorld", "@CBS", "@CNN","@FoxNews", "@nytimes"]
x_axis = 
plt.bar(users,,alpha=0.5,align="edge")
tick_locations = [value+0.4 for value in x_axis]
plt.xticks(tick_locations.["@BBCWorld", "@CBS", "@CNN","@FoxNews", "@nytimes"])
plt.xlim(-0.25, len(x_axis))
plt.ylim(0, -1)
#add chart elements
plt.title("Polarity of Tweets")
plt.xlabel("Media Source")
plt.ylabel("Overall Media Sentiment based on Twitter")

plt.savefig("sentiment_overall.png")
plt.show()